In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from dataprep.clean import clean_country
from scipy import stats
pd.options.mode.chained_assignment = None

#custom modules
from DataProcTools import Features
from DataProcTools.DataCleaning import Outliers
from DataProcTools.DataCleaning import MissingValues

### Loading the Dataset and Exploring

In [2]:
dataset = pd.read_csv('./data/athlete_events.csv')

# Initial dataset info before imputation
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Filling The Gap : imputing missing values
### Steps take to impute the data ( broken down by cell ):
Note that all the actual function implementation are done in a separate python file and the methods are called here. Please refer to that file to look at the implementation
- We first stat by describing our data to see which columns have null values and we determine that Age, Height, Weight, and Medals all have null values.
- The next step is to find the percentages of these null values in order to apply a proper imputation technique.
- We Learn that Age has 3% missing values, Height & Weight have 22% and Medals a whopping 85% missing rate
- The Age, Height and Weight are most likely MACR data due to issue with data gathering of the dataset
- The Medal is MNAR because most people that enter the olympics don't win a medal

In [3]:
missingValuesInfo = MissingValues.get_nan_values_percentage(dataset)
print(dataset.describe())
print()
print("Missing values Percentages: ", missingValuesInfo)

                  ID            Age         Height         Weight  \
count  271116.000000  261642.000000  210945.000000  208241.000000   
mean    68248.954396      25.556898     175.338970      70.702393   
std     39022.286345       6.393561      10.518462      14.348020   
min         1.000000      10.000000     127.000000      25.000000   
25%     34643.000000      21.000000     168.000000      60.000000   
50%     68205.000000      24.000000     175.000000      70.000000   
75%    102097.250000      28.000000     183.000000      79.000000   
max    135571.000000      97.000000     226.000000     214.000000   

                Year  
count  271116.000000  
mean     1978.378480  
std        29.877632  
min      1896.000000  
25%      1960.000000  
50%      1988.000000  
75%      2002.000000  
max      2016.000000  

Missing values Percentages:  {'Age': 3.494, 'Height': 22.194, 'Weight': 23.191, 'Medal': 85.326}


- The imputations techniques for each column is as follows:
- Age: Frequent Category Imputation where we replace the Ages with the mode and substitue the nulls for that ( this is because the null value are less than 5% range )
- Height, Weight: We fill those columns up using Multivariate Imputation that is inferred from the average Height and Weight per NOC ( country ) and we substitue this data in.
- Medals: Arbitrary Value Imputation was used where we just replace each occurance of 'NaN' with 'No_Medal' to indicate that no medal was won. This is possible since the data isNOT missing at random.
- We display that dataset info and head after imputation
- All implementations of this are in the Missing_Values.py file in the DataProc folder
- We didn't remove the outliers because Dr Nada send an email allowing us to skip this step

In [ ]:
# Average age per NOC ( country ) for every team entry
dataset[['Age','NOC']].groupby('NOC').Age.apply(lambda x: x.mode())

# Get average height and weight based & groupby NOC
average_height = dataset[['Height','NOC']].groupby('NOC').mean()
average_weight = dataset[['Weight','NOC']].groupby('NOC').mean()

# Get average age & store as float64
average_age = dataset['Age'].mode().loc[0]

MissingValues.arbitrary_value_imputer(dataset, 'Age', average_age)
MissingValues.arbitrary_value_imputer(dataset, 'Medal', 'no_medal')
MissingValues.height_imputer(dataset, average_height)
MissingValues.weight_imputer(dataset, average_weight)

In [ ]:
# Imputed dataset ( outliers still present )
dataset.info()
dataset.head()

### Handling Outliers
We remove the outliers by calculating the IQR score for Age, Height, Weight (numerical type data)

In [ ]:
age_outliers,height_outliers,weight_outliers = Outliers.get_outliers(dataset[['Age','Height','Weight']])


outliers_ind=pd.Index.union(weight_outliers.index,height_outliers.index)
outliers_ind=pd.Index.union(outliers_ind,age_outliers.index)

dataset_no_outliers = dataset.drop(outliers_ind)


# Research Questions Explored Below

### Question 1: Relationship between obesity rates in participants and NOC ( country )

#### Steps to get to reach an answer:
- We create a dataset with country ( NOC ), and other elements needed for BMI & BF formula calculations
- We create a gender splitting functions that shows data either for Males / Females
- We create a BMI calculating lambda function with the formula (weight / height^2)
- We create a BodyFat lambda function with the formula ( (1.2 x BMI) + (0.23 x Age) - 16.2 {if female it's 5.4} )
- We separate the male and female dataframes
- We apply the functions to both males and females then display resulting dataframes descendingly

In [ ]:
# dataframe with values needed
avg_shwn_df = dataset[['NOC','Sex','Height','Weight','Age']]

# Function to return dataset with only males / females based on input
gender_split = lambda df:(df[df.Sex=='M'],df[df.Sex=='F'])

# calculating BMI for males / females
# splitting & calc BMI   
males_df,females_df = gender_split(avg_shwn_df)
males_df['BMI'] = Features.calc_BMI(avg_shwn_df['Weight'],avg_shwn_df['Height'] )
females_df['BMI'] = Features.calc_BMI(avg_shwn_df['Weight'],avg_shwn_df['Height'] )


# Calc AvgBodyFat Percentage
males_df['AvgBF%'] = males_df.apply(Features.calc_body_fat,axis=1)
females_df['AvgBF%'] = females_df.apply(Features.calc_body_fat,axis=1)

# Displaying outputs
print("Male Average")
display(males_df.groupby('NOC').mean().sort_values('AvgBF%',ascending=False))
print("Female Average")
display(females_df.groupby('NOC').mean().sort_values('AvgBF%',ascending=False))

#### Plotting Average Height, Weight, Body Fat for males & females
we're printing a histogram for the heights of makes and females. The first 3 refer to males and the later 3 refer to the females

In [ ]:
chart_m = males_df.groupby('NOC').mean().sort_values('AvgBF%',ascending=False)
chart_f = females_df.groupby('NOC').mean().sort_values('AvgBF%',ascending=False)
# Plot Average Height, Weight, BodyFat
for col in ['Height', 'Weight', 'AvgBF%']:
    fig,axes = plt.subplots(1,2,figsize=(18,9))
    fig.suptitle(f"{col} distribution")
    sns.histplot(ax=axes[0],data=chart_m[col]).set_title("males")
    sns.histplot(ax=axes[1],data=chart_f[col]).set_title("females")

### Question 2: What is the the ratio of men to women as well as the top 10 most popular sports for men and women ?
#### Steps taken to reach the answer:
- We first create a summerData dataframe and use it for getting the ration of men vs women
    - We chose to do this instead of using both summer and winter data because prior to 1996 all of the games were performed    sequentially but this changed to the 4 year cycle that we know today and in the visualizations causes a weird up and down behaviour the gives a wrong impression of the data. We decided to plot both summer and winter and found out from the plot that more games were held in the summer and that the summer data SPECIFICALLY for this part of the question was more descripitive as the graph as more variation and ups and downs, so we chose to omit the winter data and only use the summer data. It is to be noted though that both winter and summer data show the same general trend. We only did this for ratio calculation but not for the top 10 since it wasn't needed.
- We then grouped the male female data by year and sex, get the size of it and then plot it using sns.lineplot
- to get the data for top 10 games, we use the normal dataset that is filtered based on sex so we can get male/female games and stored in a player male female dataframe.
- we then take this dataframe and group all the results by their size and sort descendingly and display the first 10.
- we display the result below for males and females in a tabular format as a panada series

#### We're also plotting the results as a barplot below. The plots are self descriptive

In [ ]:
# Summer dataframe init
summer_df = dataset[dataset['Season'] == 'Summer']

# Function to calculate ratio of men to women
def ratio_men_women(dataset):
    mf_summer = dataset.groupby(['Year', 'Sex']).size().unstack()
    mf_chart = sns.lineplot(data=mf_summer)
    return mf_chart

# Function to find top 10 events by gender
def events_by_gender(dataset, sex):
    player_mf = dataset[dataset['Sex'] == sex]
    chart_mf = player_mf.groupby('Event').size().sort_values(ascending= False).head(10)
    return chart_mf

# Display & output of functions
print('Mens Top 10 games')
male_top_10= events_by_gender(dataset, 'M')
sns.barplot(x=male_top_10.values, y=male_top_10.index)
plt.legend([],[], frameon=False) # removes legend
plt.show()

print('Womens Top 10 games')
female_top_10= events_by_gender(dataset, 'F')
sns.barplot(x=female_top_10.values, y=female_top_10.index)
plt.legend([],[], frameon=False) # removes legend
plt.show()

print('Ratio of Men to Women')
ratio_men_women(summer_df)





## Milestone_2: Noc_Regions Integration:

In [ ]:
noc_df = pd.read_csv('./data/noc_regions.csv')

display(noc_df.info())
display(noc_df.head(10))
noc_df[noc_df['region'].isna()]

#### Handling NaN Values in NOC csv file
The NOC CSV file has a 91% missing percentage for the notes entries. This data is MNAR since it's referring to a team that is formed based on a province / sub country within the main region it belongs to ( hong_kong & china for example ). We'll just replace all the missing values with a static value saying ( region_is_noc ) to show that the NOC is referring to the entire region for that entry and not a province / sub team. We'll do this using the arbitrary value substitution technique like we did above for medals.
###### Notes:
We'll fill the NaN values in region with their corresponding Notes since they're referring to either what region the team is from or if they're an independant team (IOC Refugee Olympic Team ) 

In [ ]:
# Percentage of missing NaN values:
missingValues = MissingValues.get_nan_values_percentage(noc_df)
missingValues

In [ ]:
# Notes column imputation (implementation in missingValues.py file)
MissingValues.arbitrary_value_imputer(noc_df, 'notes', 'region_is_noc')

# Imputing region based on notes
MissingValues.region_imputer(noc_df, 'notes', 'region')

In [ ]:
# Merging Noc_regions.csv with athlete_events.csv 
# Note: We drop regions since NOC has more regions that aren't included in the noc_regions.csv file 
# and we won't face any data loss.
merged_df = dataset.merge(noc_df, how='outer', on='NOC').drop(columns='region').rename(columns={'notes': 'Notes'})
merged_df.info()
merged_df.head(10)

In [ ]:
# Handling NaN values resulting from athlete_events.csv having more regions than noc_regions.csv
MissingValues.arbitrary_value_imputer(merged_df, 'Notes', 'no_notes')
merged_df.info()
display(merged_df.head())

### Integrating the "Medals.xlsx" dataset

In [ ]:
# Loading the dataset
medals_df = pd.read_excel('./data/Medals.xlsx')
medals_df.sort_values('Total', ascending=False).head(10)
medals_df.rename(columns={'Team/NOC': 'Team'}, inplace=True)
medals_df = clean_country(medals_df,'Team' ,output_format="alpha-3")
medals_df['Year'] = 2021
medals_df.rename(columns={'Team_clean': 'NOC'}, inplace=True)
medals_df.drop(columns=['Rank by Total','Rank'], axis=1, inplace=True)
medals_df.head(40)

In [ ]:
# medals_df[medals_df['Team'] == 'United States of America']

In [ ]:
#total_merge_df = merged_df.merge(medals_df, how='outer', on='Team')
total_merge_df= merged_df.drop(columns=['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Games', 'Season', 'City', 'Sport', 'Event', 'Notes'], axis= 1)
total_merge_df.info()
medals_noc_df = pd.concat([total_merge_df, medals_df],ignore_index=True)
MissingValues.arbitrary_value_imputer(medals_noc_df, 'Medal', 'no_medal')
medals_noc_df.head(10)

MissingValues.arbitrary_value_imputer(medals_noc_df, 'Gold', 0)
MissingValues.arbitrary_value_imputer(medals_noc_df, 'Silver', 0)
MissingValues.arbitrary_value_imputer(medals_noc_df, 'Bronze', 0)
MissingValues.arbitrary_value_imputer(medals_noc_df, 'Total', 0)
medals_noc_df.head(10)

for row in medals_noc_df.index:
    if(medals_noc_df['Medal'][row] == 'Gold'):
        medals_noc_df['Gold'][row] +=1
        medals_noc_df['Total'][row] +=1
    if(medals_noc_df['Medal'][row] == 'Silver'):
        medals_noc_df['Silver'][row] +=1
        medals_noc_df['Total'][row] +=1
    if(medals_noc_df['Medal'][row] == 'Bronze'):
        medals_noc_df['Bronze'][row] +=1
        medals_noc_df['Total'][row] +=1


In [ ]:
medals_noc_df.sort_values('Year', ascending=True).head(20)

grouped = medals_noc_df.sort_values(['Year'], ascending=False).groupby(['Year','Medal']).size().to_frame()
pd.reset_option('^display.', silent=True)

ax = grouped.unstack(level=0).plot(kind='bar', figsize=(9, 7))
plt.show()

In [ ]:
medals_noc_df.to_csv("data/medals_noc_integrated.csv")